In [ ]:
import numpy as np
from sklearn import datasets
import theano
import theano.tensor as T
import matplotlib.pyplot as plt
import matplotlib

# Use float32 as the default float data type
theano.config.floatX = 'float32'
# Display plots inline and change default figure size
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (5.0, 4.0)

In [ ]:
n_samples = 100
n_features = 2
n_classes = 3
train_X, train_y = datasets.make_classification(n_samples=n_samples,
                                                n_features=n_features,
                                                n_redundant=0,
                                                n_informative=2,                             
                                                n_clusters_per_class=1, 
                                                n_classes=n_classes)
train_y_onehot = np.eye(n_classes)[train_y]
plt.scatter(train_X[:, 0], train_X[:, 1], marker='o', c=train_y)

In [ ]:
# Define model
X = theano.shared(train_X.astype('float32'))
y = theano.shared(train_y_onehot.astype('float32'))

W = theano.shared(np.random.randn(n_features, n_classes).astype('float32'), name='W')
b = theano.shared(np.zeros(n_classes).astype('float32'), name='b')

z = X.dot(W) + b
y_hat = T.nnet.softmax(z)
prediction = T.argmax(y_hat, axis=1)

reg_lambda = np.float32(0.01)
loss1 = T.nnet.categorical_crossentropy(y_hat, y).mean() # use cross-entropy loss for softmax classification
loss2 = reg_lambda/2.0 * T.sum(T.sqr(W)).mean() # add L2-regularization
loss = loss1 + loss2

In [ ]:
# Gradient descent
dW = T.grad(loss, W)
db = T.grad(loss, b)

epsilon = np.float32(0.01)
gradient_step = theano.function([],
                                updates=((W, W - epsilon * dW),
                                         (b, b - epsilon * db)))

# Define loss function in order to access shared variable
f_loss = theano.function([], loss)

In [ ]:
# build model
n_epochs = 1000
for i in xrange(n_epochs):
    gradient_step()
    if i % 100 == 0:
        print i, 'loss=',f_loss()

In [ ]:
f_prediction = theano.function([], prediction, allow_input_downcast=True)

In [ ]:
y_pred = f_prediction()
plt.title("Predictions")
plt.scatter(train_X[:, 0], train_X[:, 1], marker='o', c=y_pred)

In [ ]:
error = np.mean(np.abs(np.sign(train_y-y_pred)))

In [ ]:
print 'Mean error: ', error